In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import mlflow
import mlflow.sklearn
import warnings
warnings.filterwarnings('ignore') # To suppress common warnings

In [ ]:
# always set the tracking uri at start of every session else the model and metrics will not logged on mlflow
mlflow.set_tracking_uri("http://127.0.0.1:5000")

In [13]:
# --- Start an MLflow Experiment ---
mlflow.set_experiment("Classification_Model_Comparison")

2025/11/08 04:30:21 INFO mlflow.tracking.fluent: Experiment with name 'Classification_Model_Comparison' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/813767262897942267', creation_time=1762556421509, experiment_id='813767262897942267', last_update_time=1762556421509, lifecycle_stage='active', name='Classification_Model_Comparison', tags={}>

In [2]:
df_train = pd.read_csv("df_train_preprocessed.csv")
df_test = pd.read_csv("df_test_preprocessed.csv")

x_train = df_train.drop(columns='price_2')
y_train = df_train['price_2']
x_test = df_test.drop(columns='price_2')
y_test = df_test['price_2']

In [ ]:
# preprocess the Target data - encode them to be binary than leaving it as label. XGBclassifier needs it in the binary format

In [8]:
MODELS = {
    "LogisticRegression": (LogisticRegression(solver='liblinear', random_state=42), {'C': [0.01, 0.1, 1.0]}),
    "DecisionTree": (DecisionTreeClassifier(random_state=42), {'max_depth': [3, 5, 7]}),
    "RandomForest": (RandomForestClassifier(random_state=42), {'n_estimators': [50, 100], 'max_depth': [5, 10]}),
    # "XGBoost": (XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42), {'n_estimators': [50, 100], 'learning_rate': [0.01, 0.1]}),
    "NeuralNetwork": (MLPClassifier(max_iter=1000, random_state=42), {'hidden_layer_sizes': [(50,), (100,)], 'alpha': [0.0001, 0.001]})
}

In [5]:
def log_metrics(y_true, y_pred):
    """Calculates and logs standard classification metrics."""
    metrics = {
        "accuracy_score": accuracy_score(y_true, y_pred),
        "precision_score": precision_score(y_true, y_pred, average='weighted'),
        "recall_score": recall_score(y_true, y_pred, average='weighted'),
        "f1_score": f1_score(y_true, y_pred, average='weighted')
    }
    mlflow.log_metrics(metrics)
    return metrics['f1_score']

In [14]:
best_f1_score = -1
best_model_name = ""

for model_name, (model, param_grid) in MODELS.items():
    with mlflow.start_run(run_name=model_name):
        print(f"--- Training and Tuning {model_name} ---")

        # 1. Hyperparameter Tuning using GridSearchCV
        grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, scoring='f1_weighted', n_jobs=-1)
        grid_search.fit(x_train, y_train)

        # The best model from the grid search
        best_model = grid_search.best_estimator_

        # 2. Prediction
        y_pred = best_model.predict(x_test)

        # 3. MLflow Logging
        # Log best parameters found by GridSearchCV
        mlflow.log_params(grid_search.best_params_)

        # Log Evaluation Metrics
        current_f1 = log_metrics(y_test, y_pred)

        # Log the model (stores the model artifact)
        mlflow.sklearn.log_model(best_model, "model")
        
        # Log a tag to identify the model type
        mlflow.set_tag("Model Type", model_name)

        # 4. Track the Overall Best Model
        if current_f1 > best_f1_score:
            best_f1_score = current_f1
            best_model_name = model_name
            
print(f"\n✅ All models trained and logged to MLflow.")
print(f"⭐ The best performing model based on F1-score is: **{best_model_name}** with F1-score: **{best_f1_score:.4f}**")

--- Training and Tuning LogisticRegression ---


2025/11/08 04:30:39 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/08 04:30:44 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run LogisticRegression at: http://127.0.0.1:5000/#/experiments/813767262897942267/runs/c90ad80c0df24cfe93e0bf6580bf34d8
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/813767262897942267
--- Training and Tuning DecisionTree ---


2025/11/08 04:30:46 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/08 04:30:51 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run DecisionTree at: http://127.0.0.1:5000/#/experiments/813767262897942267/runs/367a740d0c2748788dc167e266eaa4cb
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/813767262897942267
--- Training and Tuning RandomForest ---


2025/11/08 04:31:09 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/08 04:31:14 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run RandomForest at: http://127.0.0.1:5000/#/experiments/813767262897942267/runs/d7987be6a6134c3c93a91e8559a534e1
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/813767262897942267
--- Training and Tuning NeuralNetwork ---


2025/11/08 04:32:39 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/08 04:32:44 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run NeuralNetwork at: http://127.0.0.1:5000/#/experiments/813767262897942267/runs/2a382c02719241e884af1fc0651445f9
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/813767262897942267

✅ All models trained and logged to MLflow.
⭐ The best performing model based on F1-score is: **DecisionTree** with F1-score: **1.0000**


In [ ]:
#for loading mode --- may take some time

run_id = '367a740d0c2748788dc167e266eaa4cb'
logged_model_uri = f'runs:/{run_id}/model' # Replace run_id with the ID of the best run
loaded_model = mlflow.sklearn.load_model(logged_model_uri)

In [5]:
# we can also use the pyfunc to load the models --- faster
model_info_to_load = "mlflow-artifacts:/813767262897942267/models/m-c15f8d8cf32b42e18a0cb6a40f2a702d/artifacts"
loaded_model = mlflow.pyfunc.load_model(model_info_to_load)

In [9]:
predictions = loaded_model.predict(x_test)
result = pd.DataFrame(x_test)
result['actual class'] = y_test
result['predicted class'] = predictions 

In [11]:
result.tail()

,Unnamed: 0,model_photography,page,price_transformed,model_target_encoded,country_9,country_24,country_29,country_46,country_RARE_GROUP,...,colour_13,colour_14,location_1,location_2,location_3,location_4,location_5,location_6,actual class,predicted class
26471,26471,0,2,5.872616,1.0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,1,1
26472,26472,0,1,5.315813,2.0,0,0,1,0,0,...,0,0,1,0,0,0,0,0,2,2
26473,26473,1,1,5.080116,2.0,0,0,1,0,0,...,0,0,0,1,0,0,0,0,2,2
26474,26474,0,1,5.529871,1.0,0,0,1,0,0,...,1,0,0,0,1,0,0,0,1,1
26475,26475,1,3,4.519170,2.0,1,0,0,0,0,...,0,1,0,1,0,0,0,0,2,2


In [ ]:
# --- Start an MLflow Experiment ---
mlflow.set_experiment("Regression_Model_Comparison")

In [ ]:
# Next Step
# 
# Train and log the regression model on ML flow
# Perform unsupervised learning and log them on ML flow as well
# build stream lit app
# create pipe lines

# Learning and Knowledge gain:
# Deepen the understanding on each model, their hyper parameters and grid search
# Learn to interpret the MLflow graphs and make the most use of MLflow